In [ ]:
# author: Reuben Beeler (unless you don't like it--then it's anonymous)

import os
from dotenv import load_dotenv
from openai import OpenAI
from pprint import pprint

load_dotenv()

# CHOOSE YOUR MODEL (I used Groq with free tier)
llm = OpenAI(api_key=os.getenv('GROQ_API_KEY'), base_url="https://api.groq.com/openai/v1")
model_name = "llama-3.1-8b-instant"

# CHOOSE YOUR PRODUCT
product = "Wet Socks"

# NOW THE DIRTY WORK
initial_prompt = f"Make up an **original** business slogan supporting a new brand of {product}. It must be original. Only write one slogan and nothing else (keep it short). Do not wrap it in quotes."
get_validation_prompt = lambda phrase: f"Is this an original slogan? IMPORTANT: Only answer with either YES or NO without punctuation\n\n{phrase}"
get_fixit_prompt = lambda phrase: f"This slogan is not original. \n\n{phrase}\n\nModify it to be MORE original! It must be an original slogan for a company pioneering{product}. Only write one slogan and nothing else (keep it short). Do not wrap it in quotes."

verified = False
for i in range(5):
	phrase = llm.chat.completions.create(model=model_name, messages=[{"role": "user", "content": initial_prompt if i == 0 else get_fixit_prompt(phrase)}]).choices[0].message.content
	print(f"Attempt {i+1}: {phrase}")
	validation_response = llm.chat.completions.create(model=model_name, messages=[{"role": "user", "content": get_validation_prompt(phrase)}])
	validation_answer = validation_response.choices[0].message.content
	if validation_answer not in ("YES", "NO"):
		import sys
		print(f"Invalid response from validation model: {validation_answer}", file=sys.stderr)
		continue
	elif validation_answer == "YES":
		verified = True
		break

if verified:
	print(f"✅ Verified after {i+1} attempts")
else:
	print(f"❌ Failed to verify after {i+1} attempts")

EXAMPLE_OUTPUT = """
Attempt 1: Wet Steps Welcome Everything
Attempt 2: Socks Soaked in Storytelling
Attempt 3: Warming Hearts, One Wet Sock at a Time
Attempt 4: Stepping into comfort, one dry pair at a time.
Attempt 5: Drying the moment, one soggy step at a time.
✅ Verified after 5 attempts
"""

Attempt 1: Wrap your feet in comfort, soak your soul in nostalgia.
Attempt 2: Step into the warmth that time forgot.
Attempt 3: Slip into the comfort of yesterday's footprints.
Attempt 4: Wear the memory, feel the history.
Attempt 5: Step into the damp echo of yesterday.
❌ Failed to verify after 5 attempts
